# Data Prep
# Stage 1 : Data for subreddit graph (69 cases -> words)
# Stage 2 : Vocabulary and Co-occurence (words -> cooccurence)
# Stage 3 : Twitter Data
# Stage 4 : More Co-occurence (from Twitter)

# Stage 5 : Rank Cooccurence with #Appearances (rank(cooccurence))
# Stage 6 : Rank Cooccurence with Retweets (rank(cooccurence))

# Query Part 1: Given Query -> Cooccurence Ranking

In [141]:
from google.cloud import bigquery
from google.oauth2 import service_account

# TODO(developer): Set key_path to the path to the service account key
#                  file.
key_path = "../nwo-sample-5f8915fdc5ec.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

# reddit data

In [193]:
import nltk
import pickle
import statistics
import numpy as np
from itertools import permutations  
from itertools import combinations 
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
from string import punctuation

def Query_Processor(query):
    # input: sql query
    # return: rows of data
    QUERY = query
    query_job = client.query(QUERY)  # API request
    rows = query_job.result()  # Waits for query to finish
    return rows

def Tokenize_Remove_Stop_Words(Sentence):
    # input: Sentence as String
    # return: New Word List without stop words
    stop_words = set(stopwords.words('english') + list(punctuation) + ["gt","amp","s=y","▪︎","n't","'s",'--','https','”', '“', '’', '…', '’',"'re","''","``","...","**"])  
    word_tokens = word_tokenize(Sentence)  
    New_Word_list = [w for w in word_tokens if not w.lower() in stop_words]  
    return New_Word_list

'''
def xy():
    # input: 
    # return: 
    return 
'''

'\ndef xy():\n    # input: \n    # return: \n    return \n'

# subreddit graph (69 cases)

In [194]:
# Perform a query.
#QUERY = (
#    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
#    'WHERE state = "TX" '
#    'LIMIT 100')

# Querying data
query = (
    'SELECT subreddit, count(subreddit) FROM `nwo-sample.graph.reddit` '
    'GROUP BY subreddit')
rows = Query_Processor(query)

new_list_subreddit = []
for row in rows:
    #print(list(row))
    new_list_subreddit.append(list(row))
    #print(list(row)[0],list(row)[1])
    
print(len(new_list_subreddit))
print (new_list_subreddit)

69
[['IndiaInvestments', 183807], ['TechNewsToday', 313738], ['Skincare_Addiction', 75243], ['unitedkingdom', 2397047], ['WayOfTheBern', 1581583], ['UpliftingNews', 1603851], ['news', 2589336], ['worldnews', 2807377], ['AsianBeauty', 794347], ['technology', 2621200], ['nyc', 1605285], ['CryptoCurrency', 2155943], ['business', 343106], ['PersonalFinanceNZ', 118294], ['MachineLearning', 301079], ['Daytrading', 184711], ['politicaldiscussions', 75], ['finance', 155607], ['Sephora', 92729], ['immigration', 222234], ['gunpolitics', 542692], ['moderatepolitics', 728599], ['eupersonalfinance', 64211], ['RenewableEnergy', 40038], ['AusSkincare', 67864], ['30PlusSkinCare', 103403], ['btc', 1690096], ['RobinHood', 376485], ['lgbt', 1551666], ['PanPorn', 337891], ['geopolitics', 445647], ['RealEstate', 819289], ['UKPersonalFinance', 973766], ['startups', 345904], ['RobinHoodPennyStocks', 356557], ['glossier', 119453], ['RefundsForDays', 12182], ['SkincareAddicts', 105492], ['ukpolitics', 2607002]

# Stage 2a : Vocabulary 

In [195]:
# Selecting Vocabulary
all_subreddit_vocabulary_selection = {}
for subreddit in new_list_subreddit:
    print(subreddit[0])
    query = (
    'SELECT body FROM `nwo-sample.graph.reddit` '
    'WHERE subreddit = "'+subreddit[0]+'"')
    rows = Query_Processor(query)
    
    vocabulary_selection = {}
    count_print = 0
    for row in rows:
        #print ((list(row)))
        Sentence = list(row)[0]
        tokens = Tokenize_Remove_Stop_Words(Sentence)
        for token in tokens:
            if vocabulary_selection.get(token) == None:
                vocabulary_selection[token] = 1
            else:
                vocabulary_selection[token] += 1
        #print(tokens)
        if count_print == 2000: break # restricting the number of rows
        else: count_print+=1
    
    # Selecting Top X% or Top 2K
    print ("Total Vocabulary ",len(vocabulary_selection))
    count_v = 0
    new_selected_vocabulary_selection = {}
    for key, value in sorted(vocabulary_selection.items(), key=lambda item: item[1], reverse=True):
        new_selected_vocabulary_selection[key] = value
        #print("%s: %s" % (key, value))
        if count_v == 2000: break # restricting the number of words in vocabulary to 2K
        else: count_v+=1
    all_subreddit_vocabulary_selection[subreddit[0]] = new_selected_vocabulary_selection
    
    #break

IndiaInvestments
Total Vocabulary  8932
TechNewsToday
Total Vocabulary  9414
Skincare_Addiction
Total Vocabulary  8512
unitedkingdom
Total Vocabulary  10617
WayOfTheBern
Total Vocabulary  13027
UpliftingNews
Total Vocabulary  9370
news
Total Vocabulary  10462
worldnews
Total Vocabulary  11016
AsianBeauty
Total Vocabulary  9846
technology
Total Vocabulary  10901
nyc
Total Vocabulary  9665
CryptoCurrency
Total Vocabulary  9016
business
Total Vocabulary  12462
PersonalFinanceNZ
Total Vocabulary  9965
MachineLearning
Total Vocabulary  12436
Daytrading
Total Vocabulary  7564
politicaldiscussions
Total Vocabulary  1172
finance
Total Vocabulary  10020
Sephora
Total Vocabulary  6920
immigration
Total Vocabulary  8313
gunpolitics
Total Vocabulary  12596
moderatepolitics
Total Vocabulary  15606
eupersonalfinance
Total Vocabulary  10376
RenewableEnergy
Total Vocabulary  11478
AusSkincare
Total Vocabulary  9608
30PlusSkinCare
Total Vocabulary  9727
btc
Total Vocabulary  10267
RobinHood
Total Vocab

# Stage 2b : Co-occurence (subreddit -> cooccurence)

In [196]:
# Generate Cooccurence
all_subreddit_cooccur_selection = {}
for subreddit in new_list_subreddit:
    print(subreddit[0])
    query = (
    'SELECT body FROM `nwo-sample.graph.reddit` '
    'WHERE subreddit = "'+subreddit[0]+'"')
    rows = Query_Processor(query)
    
    cooccur_selection = {}
    count_print = 0
    for row in rows:
        #print ((list(row)))
        Sentence = list(row)[0]
        tokens = Tokenize_Remove_Stop_Words(Sentence)
        tokens_perm = combinations(tokens, 2) 
        for token in tokens_perm:
            #print (sorted(list(token)))
            temp = sorted(list(token))
            if temp[0]==temp[1]: continue
            if cooccur_selection.get(temp[0]+','+temp[1]) == None:
                cooccur_selection[temp[0]+','+temp[1]] = 1
            else:
                cooccur_selection[temp[0]+','+temp[1]] += 1
        #print(tokens)
        if count_print == 2000: break # restricting the number of rows
        else: count_print+=1
    
    
    # Selecting Top X% or Top 2K
    print ("Total Co-occurences ",len(cooccur_selection))
    count_v = 0
    new_selected_cooccur_selection = {}
    for key, value in sorted(cooccur_selection.items(), key=lambda item: item[1], reverse=True):
        new_selected_cooccur_selection[key] = value
        #print("%s: %s" % (key, value))
        if count_v == 2000: break # restricting the number of cooccurences to 2K
        else: count_v+=1
    
    all_subreddit_cooccur_selection[subreddit[0]] = new_selected_cooccur_selection
    
    #break

IndiaInvestments
Total Co-occurences  688883
TechNewsToday
Total Co-occurences  769859
Skincare_Addiction
Total Co-occurences  895398
unitedkingdom
Total Co-occurences  644730
WayOfTheBern
Total Co-occurences  1240091
UpliftingNews
Total Co-occurences  545179
news
Total Co-occurences  775672
worldnews
Total Co-occurences  1019418
AsianBeauty
Total Co-occurences  880043
technology
Total Co-occurences  843861
nyc
Total Co-occurences  589584
CryptoCurrency
Total Co-occurences  631533
business
Total Co-occurences  952350
PersonalFinanceNZ
Total Co-occurences  1216720
MachineLearning
Total Co-occurences  1349310
Daytrading
Total Co-occurences  636520
politicaldiscussions
Total Co-occurences  36601
finance
Total Co-occurences  703040
Sephora
Total Co-occurences  484252
immigration
Total Co-occurences  750149
gunpolitics
Total Co-occurences  1534698
moderatepolitics
Total Co-occurences  3244183
eupersonalfinance
Total Co-occurences  1173605
RenewableEnergy
Total Co-occurences  1067915
AusSkin

# Stage 5 : Rank Cooccurence with #Appearances (rank(cooccurence))

In [197]:
# The Top Co-occurences for each subreddit

#all_subreddit_cooccur_selection['Bitcoin']
#all_subreddit_cooccur_selection['IndiaInvestments']
#all_subreddit_cooccur_selection['Coronavirus']
all_subreddit_cooccur_selection['DisneyPlus']
#all_subreddit_cooccur_selection['nyc']
#all_subreddit_cooccur_selection['SkincareAddicts']

{'Disney,Hulu': 243,
 'Disney,would': 123,
 'Disney,Disney+': 121,
 'Disney,content': 110,
 'Disney+,Hulu': 78,
 'Disney+,would': 77,
 'Disney,get': 76,
 'Disney,one': 75,
 'Hulu,would': 75,
 'content,would': 68,
 'content,like': 65,
 'Disney,like': 60,
 'Disney,shows': 60,
 'Hulu,content': 60,
 'Disney+,content': 58,
 'Disney,agreement': 56,
 'Comcast,Disney': 55,
 'Disney,really': 54,
 'Hulu,two': 54,
 'Disney,service': 54,
 'content,get': 53,
 'Disney,people': 52,
 'Disney,already': 52,
 'Comcast,Hulu': 52,
 'like,movie': 50,
 'like,would': 50,
 'Hulu,services': 49,
 'content,shows': 49,
 'Disney+,get': 48,
 'Star,Wars': 48,
 'Disney,movies': 48,
 'film,release': 48,
 'Disney,original': 47,
 'get,would': 47,
 'like,really': 45,
 'Disney,Star': 45,
 'Disney,TV': 44,
 'Disney,app': 44,
 'Disney,think': 44,
 'Disney,services': 44,
 'Hulu,agreement': 44,
 'get,like': 43,
 'Disney,two': 43,
 'Disney,Plus': 43,
 'one,would': 42,
 'Disney,pay': 42,
 'Disney,year': 42,
 'like,one': 41,
 'kn

# twitter data

# Top Twitter Handles

In [198]:
# Perform a query.
#QUERY = (
#    'SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` '
#    'WHERE state = "TX" '
#    'LIMIT 100')

# Querying 
query = (
    'SELECT * '
    'FROM (SELECT name, count(name) as temp '
    'FROM `nwo-sample.graph.tweets` '
    'GROUP BY name) where temp > 21000 ')
rows = Query_Processor(query)

new_list_tweeter_name = []
for row in rows:
    #print(list(row))
    new_list_tweeter_name.append(list(row))
    #print(list(row)[0],list(row)[1])
    
print(len(new_list_tweeter_name))
print (new_list_tweeter_name)

362


[['BitcoinAgile', 36102], ['elisabeth', 22006], ['Chloe', 22964], ['Simon', 23742], ['Amber', 38220], ['Michael Robert Lawrence is busy AF! ✍️😮🖥️📚☕️', 24354], ['Ali', 26386], ['David, Ph.D.', 23348], ['Rebecca Lanzot', 29962], ['Brooke', 34338], ['bee', 25876], ['Avocados From Mexico', 28257], ['Sam', 29190], ['DEAL DONKEY Podcast Deals 24/7 Retro Switch PS4', 47793], ['CBS News', 35633], ['The Associated Press', 23188], ['Refinery29 UK', 21382], ['WSMV News4 Nashville', 27032], ['Styles Rant', 21522], ['Zoe', 25700], ['Rob', 46937], ['Lisa', 77837], ['Adam Singer', 22845], ['Rappler', 28636], ['Air India', 39303], ['Tata Sky', 31391], ['MadameNoire', 21366], ['Good Housekeeping', 21894], ['NBC New York', 23878], ['Molly', 29880], ['Rich Tehrani', 23027], ['Chris Taylor', 22365], ['Hayley', 29986], ['Holly', 53598], ['🐷 Daisy Mae Parker 🐷', 28817], ['Current UTC', 33324], ['Paytm Care', 22544], ['shae shukla', 21010], ['Michael Rosen', 21480], ['Elizabeth', 21365], ['Sputnik', 25429], 

# Stage 4 : More Co-occurence (from Twitter)

In [199]:
# Generate Cooccurence
#all_tweeter_cooccur_selection = {}
new_selected_cooccur_selection = {}
new_selected_cooccur_selection_retweet = {}
for tweeter in ["CNN"]: # new_list_tweeter
    print(tweeter)
    query = (
    'SELECT tweet, retweet_count '
    'FROM `nwo-sample.graph.tweets` '
    ' WHERE name = "'+tweeter+'"')
    rows = Query_Processor(query)
    
    cooccur_selection = {}
    cooccur_selection_retweet = {}
    count_print = 0
    for row in rows:
        #print (list(row)[1], type(list(row)[1]))
        Sentence = list(row)[0]
        Retweet_Count = list(row)[1]
        tokens = Tokenize_Remove_Stop_Words(Sentence)
        tokens_perm = combinations(tokens, 2) 
        for token in tokens_perm:
            #print (sorted(list(token)))
            temp = sorted(list(token))
            if temp[0]==temp[1]: continue
            if cooccur_selection.get(temp[0]+','+temp[1]) == None:
                cooccur_selection[temp[0]+','+temp[1]] = 1
                cooccur_selection_retweet[temp[0]+','+temp[1]] = Retweet_Count
            else:
                cooccur_selection[temp[0]+','+temp[1]] += 1
                #print (Retweet_Count, cooccur_selection_retweet[temp[0]+','+temp[1]])
                cooccur_selection_retweet[temp[0]+','+temp[1]] = Retweet_Count + cooccur_selection_retweet[temp[0]+','+temp[1]]
        #print(tokens)
        if count_print == 2000: break # restricting the number of rows
        else: count_print+=1
    
    # Selecting Top X% or Top 2K (Apearances)
    print ("Total Cooccurence ",len(cooccur_selection))
    count_v = 0
    for key, value in sorted(cooccur_selection.items(), key=lambda item: item[1], reverse=True):
        new_selected_cooccur_selection[key] = value
        #print("%s: %s" % (key, value))
        if count_v == 2000: break # restricting the number of cooccurences to 2K
        else: count_v+=1
            
    # Selecting Top X% or Top 2K (Retweets)
    print ("Total Cooccurence ",len(cooccur_selection_retweet))
    count_v = 0
    for key, value in sorted(cooccur_selection_retweet.items(), key=lambda item: item[1], reverse=True):
        new_selected_cooccur_selection_retweet[key] = value
        #print("%s: %s" % (key, value))
        if count_v == 2000: break # restricting the number of cooccurences to 2K
        else: count_v+=1
    
    #all_tweeter_cooccur_selection = new_selected_cooccur_selection
    
    break

CNN
Total Cooccurence  323062
Total Cooccurence  323062


# Stage 5 : Rank Cooccurence with #Appearances (rank(cooccurence))

In [200]:
new_selected_cooccur_selection

{'President,Trump': 230,
 'coronavirus,pandemic': 116,
 'Biden,Joe': 87,
 'New,York': 64,
 'US,coronavirus': 62,
 'President,says': 59,
 'coronavirus,says': 59,
 'Trump,coronavirus': 55,
 'House,White': 53,
 'Trump,says': 53,
 'Biden,President': 51,
 'President,coronavirus': 49,
 'cases,new': 47,
 'Dr.,Fauci': 46,
 'Joe,President': 45,
 'coronavirus,people': 44,
 'coronavirus,new': 44,
 'Trump,said': 44,
 'cases,coronavirus': 44,
 'according,new': 43,
 'Donald,Trump': 43,
 'Floyd,George': 43,
 'http,says': 42,
 'Trump,administration': 42,
 'Biden,Trump': 42,
 'Dr.,says': 40,
 'Trump,US': 39,
 'Covid-19,says': 38,
 'according,coronavirus': 37,
 'coronavirus,said': 37,
 'President,said': 37,
 'Anthony,Dr.': 37,
 'Anthony,Fauci': 37,
 'President,pandemic': 36,
 'Blake,Jacob': 36,
 'President,former': 35,
 'Gov,state': 33,
 'Covid-19,pandemic': 33,
 'Trump,pandemic': 33,
 'States,United': 33,
 'pandemic,says': 31,
 'coronavirus,state': 31,
 'Joe,Trump': 31,
 'Donald,President': 31,
 'Conve

# Stage 6 : Rank Cooccurence with Retweets (rank(cooccurence))

In [201]:
new_selected_cooccur_selection_retweet

{'President,Trump': 104656,
 'coronavirus,pandemic': 41739,
 'Biden,Joe': 34212,
 'Trump,coronavirus': 32262,
 'black,woman': 32069,
 'President,coronavirus': 27105,
 'Donald,Trump': 25523,
 'Trump,administration': 23906,
 'US,coronavirus': 23218,
 'Former,President': 23125,
 'Joe,President': 22450,
 'President,former': 22324,
 'President,says': 22084,
 'Biden,President': 21850,
 'Trump,said': 21361,
 'President,said': 21150,
 'Obama,President': 20892,
 'Floyd,George': 20868,
 'Blake,Jacob': 20750,
 'Dr.,Fauci': 20646,
 'Black,likely': 19628,
 'Black,doctors': 19628,
 'doctors,likely': 19628,
 'House,White': 19578,
 'New,York': 19537,
 'coronavirus,people': 19184,
 'people,police': 18936,
 'coronavirus,new': 18780,
 'Trump,former': 18761,
 'Donald,President': 18746,
 'Anthony,Fauci': 18679,
 'election,people': 18516,
 'Trump,says': 18409,
 'America,people': 18332,
 'Wisconsin,people': 18105,
 'Anthony,Dr.': 18043,
 'Trump,US': 17834,
 'calls,people': 17827,
 'according,new': 17658,
 'p

# Query Part 1: Given Query -> Nearest subreddit -> Cooccurence Ranking

In [208]:
query_item_for_search = 'coronavirus' 
query_item_for_search = 'Trump'
query_item_for_search = 'stock'
query_item_for_search = 'war'
query_item_for_search = 'wallstreet'

query_subreddit_marks = {}
for key, value in all_subreddit_cooccur_selection.items(): # key -> subreddit, value -> 
    #print (key)
    for occurence, appearance in value.items():
        if query_item_for_search in occurence:
            if query_subreddit_marks.get(key) == None:
                query_subreddit_marks[key] = 1
            else:
                query_subreddit_marks[key] += 1

print(query_subreddit_marks)

# The best subreddit is printed
print("The item searched - ",query_item_for_search)
print("The rest cooccurence ranked are as follows : ")
for key, value in sorted(query_subreddit_marks.items(), key=lambda item: item[1], reverse=True):
    print(all_subreddit_cooccur_selection[key])
    break

{'wallstreetbets': 25}
The item searched -  wallstreet
The rest cooccurence ranked are as follows : 


{'automatically,bot': 119, '*I,automatically': 118, 'action,automatically': 118, 'automatically,performed': 118, 'Please,automatically': 118, 'automatically,contact': 118, 'automatically,moderators': 118, 'automatically,subreddit': 118, '/message/compose/,automatically': 118, 'automatically,to=/r/wallstreetbets': 118, 'automatically,questions': 118, 'automatically,concerns': 118, '*I,bot': 102, 'action,bot': 102, 'bot,performed': 102, 'Please,bot': 102, 'bot,contact': 102, 'bot,moderators': 102, 'bot,subreddit': 102, '/message/compose/,bot': 102, 'bot,to=/r/wallstreetbets': 102, 'bot,questions': 102, 'bot,concerns': 102, 'questions,subreddit': 102, '*I,action': 101, '*I,performed': 101, '*I,Please': 101, '*I,contact': 101, '*I,moderators': 101, '*I,subreddit': 101, '*I,/message/compose/': 101, '*I,to=/r/wallstreetbets': 101, '*I,questions': 101, '*I,concerns': 101, 'action,performed': 101, 'Please,action': 101, 'action,contact': 101, 'action,moderators': 101, 'action,subreddit': 101, '